In [4]:
import requests, json, os, bs4
from google.auth import default
from google.auth.transport.requests import Request

In [5]:
from google.colab import drive
import json, os

# Connect with Google Drive
drive.mount('/content/drive')

# read API keys from JSON file
with open('/content/drive/MyDrive/secrets/api_keys.json', encoding='utf-8-sig') as f:
    secrets = json.load(f)

# get the keys
SERP_KEY   = secrets["SERPAPI_API_KEY"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
PROJECT_ID = secrets["PROJECT_ID"]
LOCATION = secrets["LOCATION"]
MODEL_ID = secrets["LLAMA_MODEL"]

ENDPOINT = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/" \
           f"projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi/chat/completions"

TEMPERATURE = 0.2
TOP_P = 1.0
MAX_TOKENS = 3000

In [7]:
def _get_iam_token() -> str:
    creds, _ = default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
    creds.refresh(Request())
    return creds.token

def call_llama(messages, *,
               temperature=TEMPERATURE,
               top_p=TOP_P,
               max_tokens=MAX_TOKENS) -> str:
    """OpenAI-compatible Vertex AI chat/completions call"""
    body = {
        "model": MODEL_ID,
        "messages": messages,
        "temperature": temperature,
        "top_p": top_p,
        "max_tokens": max_tokens
    }
    headers = {
        "Authorization": f"Bearer {_get_iam_token()}",
        "Content-Type":  "application/json"
    }
    r = requests.post(ENDPOINT, headers=headers, json=body, timeout=120)
    if r.status_code != 200:

        raise RuntimeError(f"{r.status_code} – {r.text}")
    return r.json()["choices"][0]["message"]["content"]


In [8]:
def serp_links(query: str, k: int = 3):
    params  = {"engine": "google", "q": query, "api_key": secrets["SERPAPI_API_KEY"]}
    results = requests.get("https://serpapi.com/search", params=params).json()
    return [r["link"] for r in results.get("organic_results", [])[:k]]

def fetch_page_text(url: str, n_chars: int = 1500):
    try:
        html = requests.get(url, timeout=10).text
        text = bs4.BeautifulSoup(html, "html.parser").get_text(" ", strip=True)
        return text[:n_chars]
    except Exception as e:
        print("Fetch error:", url, e)
        return ""

tender_links   = serp_links("security camera procurement tender site:europa.eu")
supplier_links = serp_links("security camera supplier Europe price range site:.com OR site:.co.uk")
print("🔗  Tender links:\n", tender_links)
print("🔗  Supplier links:\n", supplier_links, "\n")


🔗  Tender links:
 ['https://op.europa.eu/en/web/public-procurement/procurement-details/-/procurement/dba52083-8af6-4ab4-82a2-d7192aa0a0f7', 'https://op.europa.eu/en/web/public-procurement/procurement-details/-/procurement/37a10779-0375-4408-9236-afe3bcd36f6a', 'https://op.europa.eu/en/web/public-procurement/procurement-details/-/procurement/0d540a61-b646-4cef-97c1-e6a97ad74721']
🔗  Supplier links:
 ['https://uk.swann.com/products/?srsltid=AfmBOopChlfgI8r8CFY2R_v9a9QlRGxWW9tGsyy0EFSXs3VNPdgdpclA', 'https://www.eufy.com/collections/security-camera', 'https://www.kentfaith.com/KF50.0006AEU_outdoor-security-camera-2k-wireless-camera-for-home-security-2-4ghz-wifi-security-camera-with-auto-tracking-motion-detection-with-spotlight-color-night-vision-camera-2-way-audio-euro-gauge?srsltid=AfmBOoqgLrty_CZWoTQSocAhtoRvdl0RRuVRkMHiZHZkmT5NLoj_cEmQ'] 



In [9]:

tender_texts   = [fetch_page_text(u) for u in tender_links]
supplier_texts = [fetch_page_text(u) for u in supplier_links]

tender_prompt = f"""
You are a public tender expert for Dublin City Council.

Below are documents from security camera public tenders:
{tender_texts[0]}
{tender_texts[1] if len(tender_texts) > 1 else ''}

1. Summarize the most common **technical and legal requirements**.
2. Generate 5 clarification questions that a city official should answer before drafting a new tender.
"""

supplier_prompt = f"""
You are a market analyst advising a municipality about camera system purchases.

Below are supplier websites or documents:
{supplier_texts[0]}
{supplier_texts[1] if len(supplier_texts) > 1 else ''}

1. Summarize the most common **features, pricing, and delivery models** offered in the market.
2. Generate 5 clarifying questions to help compare or choose between supplier options.
"""

print("📑  Tender summary:\n",
      call_llama([{"role":"user","content":tender_prompt}]), "\n")
print("🛒  Supplier summary:\n",
      call_llama([{"role":"user","content":supplier_prompt}]), "\n")


📑  Tender summary:
 **1. Summary of the most common technical and legal requirements:**

Based on the provided documents, the following technical and legal requirements are likely to be common in security camera public tenders:

* Technical requirements:
	+ Cameras must be digital (IP-based)
	+ Cameras and related accessories must meet certain standards for video surveillance and protection
	+ Equipment must be compatible with existing infrastructure (not explicitly stated but implied)
* Legal requirements:
	+ Compliance with EU procurement regulations (as evidenced by the publication on the EU's Publications Office website)
	+ Adherence to EU law and relevant directives (e.g., data protection, cybersecurity)
	+ Potential requirements for supplier certification, warranty, and after-sales support

**2. Clarification questions for a city official to answer before drafting a new tender:**

1. **What are the specific security concerns or objectives that the new camera surveillance system i

In [10]:
user_answers = """(enter user answers here)"""

draft_prompt = f"""
You are a public procurement expert working at Dublin City Council.
You have conducted a market analysis on municipal surveillance camera systems and identified both common supplier offerings and typical tender requirements.

Now write a **complete and professional 'Requirements and Specifications' section** for a new security camera tender.
Structure it just like the real Irish tender documents provided before. Include the following:

- A short introduction / background
- Detailed technical requirements in bullet points or tables (e.g., resolution, night vision, weatherproofing, storage, analytics)
- Legal and compliance requirements (especially GDPR and EU procurement directives)
- Installation and maintenance responsibilities
- Service levels (SLAs), warranties, and support
- Any environmental or sustainability standards
- Pricing and contractual expectations

Make sure the structure and language matches what is commonly used in Irish public sector tenders.
Do not generate questions. Generate a final tender section text.
Length: at least 700–1000 words.

--- USER NOTES ---
{user_answers}
"""

response_text = call_llama([{"role":"user","content":draft_prompt}])
print("📄  1. draft output (first 300 chars):\n", response_text[:300], "...\n")


📄  1. draft output (first 300 chars):
 **Requirements and Specifications for Municipal Surveillance Camera System**

**Introduction/Background**

Dublin City Council is seeking to procure a municipal surveillance camera system to enhance public safety and security within the city. The proposed system will be used to monitor and record in ...



In [11]:
from google.colab import files, output
uploaded = files.upload()   # Choose sample PDF's

Saving Car Park Management Solution ITT_Final.pdf to Car Park Management Solution ITT_Final (3).pdf
Saving Final Gully Monitoring CFT.pdf to Final Gully Monitoring CFT (3).pdf
Saving RFT for Service Design.pdf to RFT for Service Design (3).pdf


In [12]:

!pip -q install PyPDF2
from PyPDF2 import PdfReader

sample_texts = []
for fn in uploaded.keys():
    reader = PdfReader(fn)
    txt = "".join(p.extract_text() or "" for p in reader.pages)[:1500]
    sample_texts.append(txt)

pdf_prompt = f"""
You are a public procurement expert working at Dublin City Council.

Here are real tender excerpts from previous Irish municipal projects:
{sample_texts[0]}
{sample_texts[1] if len(sample_texts)>1 else ''}

You have also conducted a market analysis and collected key answers:

--- USER CLARIFICATIONS ---
{user_answers}

Your task is to write an extremely comprehensive **"Requirements and Specifications"** section for a new public tender to procure security cameras and video management infrastructure.

Your response MUST:
- Be at least **1500 words** in length.
- Fully reflect the **structure, level of detail, language style**, and **subsection formatting** used in Irish public tenders (like the examples above).
- Include multiple tables (e.g., camera specs, VMS features, installation steps, SLAs, etc.).
- Use **bullet points**, **numbered sections**, **section headers**, and **clear formatting**.
- Explicitly mention relevant Irish/EU laws (e.g., GDPR, CE Marking, ISO27001, 2014/24/EU).
- Present both **mandatory** and **desirable** features, clearly marked.
- Emphasize sustainability, accessibility, cybersecurity, training, and documentation expectations.
- Cover full **lifecycle**: procurement, delivery, installation, support, exit/transition.
- **Table-layout rules:** any table must fit on an A4 page in portrait orientation with 2 cm margins;
  • limit to **max 3-4 columns**, each ≤ 6 cm wide.
  • If a cell’s text would overflow, replace it with a superscript footnote symbol (e.g. ¹) and put the full explanation in a “Table Notes” paragraph directly under the table.
  • Split very wide tables into multiple stacked tables (e.g. break after the “Verification Method” column).
- **Column & text limits:** every table may contain **max 4 columns** and **max 30 characters per cell** (including spaces).
  • Overflow → superscript note mark, explanation in “Table Notes”.
  • Split if still too wide.
  • **Do NOT reuse the sample column headings; choose headings appropriate to each table.**
  Example layout (for structure only, not wording):
  | A | B | C | D¹ |
For the “Min. Numerical Value” column, always state a concrete figure and unit (e.g., “0.01 Lux”, “IP66”); **do not use Yes/No here**—if the feature is binary, move Yes/No to a separate “Supported?” column instead.
- In every table, **do NOT leave any cell blank**.
  • If the value is the same as the row above, repeat it verbatim (e.g., write “Platform” again) or use the ditto symbol “〃”.
  • Keep the column order fixed so that each row has exactly the same number of cells.

- Insert Word-friendly formatting hints (e.g. “|---|” rows exactly match the final column count) so the table can be pasted without breaking.

Do not generate questions — only the full section text, at least 1000 words.
"""

response_text = call_llama([{"role":"user","content":pdf_prompt}])
print(response_text)


**Requirements and Specifications for Security Cameras and Video Management Infrastructure**

**1. Introduction**

Dublin City Council is seeking to procure a comprehensive security camera and video management infrastructure solution to enhance the safety and security of its citizens, staff, and infrastructure. This tender aims to provide a robust, scalable, and sustainable solution that meets the Council's current and future needs.

**2. Scope of Works**

The scope of this tender includes the provision of security cameras, video management software, and related infrastructure to support the Council's security requirements. The successful tenderer will be responsible for the delivery, installation, configuration, testing, and commissioning of the solution.

**3. Technical Requirements**

The solution must meet the following technical requirements:

### 3.1 Security Cameras

| **Camera Type** | **Min. Resolution** | **Min. Frame Rate** | **Supported?** |
| --- | --- | --- | --- |
| Fixe

In [13]:
import importlib.util, subprocess, sys, pathlib
if importlib.util.find_spec("docx") is None:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "python-docx"])

from docx import Document

doc = Document()
for line in response_text.split("\n"):
    doc.add_paragraph(line)

doc_path = "security_camera_requirements2.docx"
doc.save(doc_path)

print("✅ Word file saved.:", doc_path)


✅ Word file saved.: security_camera_requirements2.docx
